In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img



In [2]:
# Define the paths to your train and test dataset
train_dir = r"D:\PHD work\3_cls\train"
test_dir = r"D:\PHD work\3_cls\test"

# Create ImageDataGenerator instances for loading the dataset
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load train data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(176, 208),  # Resize all images to 150x150 pixels
    batch_size=32,
    class_mode='categorical'  # 3-class classification
)

# Load the training dataset
test_generator = test_datagen.flow_from_directory(
    train_dir,
    target_size=(176, 208),  # Resize images to 150x150
    batch_size=32,           # Batch size
    class_mode='categorical'  # Since you have multiple classes
)

Found 6477 images belonging to 3 classes.
Found 6477 images belonging to 3 classes.


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Multiply

# First CNN Stream
def build_cnn_stream(input_shape):
    cnn_input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(cnn_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    return cnn_input, x

# Attention Mechanism
def soft_attention(fusion_features):
    attention = Dense(128, activation='sigmoid')(fusion_features)
    attention = Dense(1, activation='sigmoid')(attention)
    return attention

# Build the Dual-Stream Model
def build_dual_stream_model(input_shape, num_classes):
    # Stream 1
    input1, stream1_features = build_cnn_stream(input_shape)
    
    # Stream 2 (could be the same architecture for simplicity)
    input2, stream2_features = build_cnn_stream(input_shape)

    # Concatenate the outputs of both streams
    fusion_features = Concatenate()([stream1_features, stream2_features])

    # Apply soft-attention to the concatenated features
    attention = soft_attention(fusion_features)
    
    # Attention-weighted features
    attended_features = Multiply()([fusion_features, attention])

    # Fully connected layer for classification
    x = Dense(128, activation='relu')(attended_features)
    output = Dense(num_classes, activation='softmax')(x)

    # Define the model
    model = Model(inputs=[input1, input2], outputs=output)
    return model


In [4]:
# Build the model
input_shape = (176, 208, 3)  # Image shape: 150x150 pixels with 3 channels (RGB)
num_classes = 3  # 3 classes for classification
model = build_dual_stream_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Show the model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 176, 208, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 176, 208, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 176, 208, 64)      │           1,792 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 176, 208, 64)      │           1,792 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 88, 104, 64)       │               0 │ conv2d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 88, 104, 64)       │               0 │ conv2d_3[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 88, 104, 128)      │          73,856 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 88, 104, 128)      │          73,856 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 44, 52, 128)       │               0 │ conv2d_1[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_4               │ (None, 44, 52, 128)       │               0 │ conv2d_4[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 44, 52, 256)       │         295,168 │ max_pooling2d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 44, 52, 256)       │         295,168 │ max_pooling2d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_2               │ (None, 22, 26, 256)       │               0 │ conv2d_2[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_5               │ (None, 22, 26, 256)       │               0 │ conv2d_5[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 146432)            │               

 Total params: 75,715,588 (288.83 MB)

 Trainable params: 75,715,588 (288.83 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
import tensorflow as tf

def dual_input_generator(generator):
    while True:
        # Fetch a batch from the original image generator (this yields a batch of images and labels)
        batch_x, batch_y = next(generator)
        
        # Convert numpy arrays to tensors for images and labels
        batch_x_tensor = tf.convert_to_tensor(batch_x, dtype=tf.float32)  # Input images
        batch_y_tensor = tf.convert_to_tensor(batch_y, dtype=tf.float32)  # One-hot encoded labels

        # Yield the batch: two identical image streams and the labels, ensure they're tuples
        yield (batch_x_tensor, batch_x_tensor), batch_y_tensor



In [6]:
def get_output_signature(image_shape, num_classes):
    return (
        (tf.TensorSpec(shape=(None, *image_shape), dtype=tf.float32),  # First image stream
         tf.TensorSpec(shape=(None, *image_shape), dtype=tf.float32)), # Second image stream
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)      # Labels
    )




In [7]:
train_generator_custom = tf.data.Dataset.from_generator(
    lambda: dual_input_generator(train_generator),
    output_signature=get_output_signature((176, 208, 3), 3)  # Image shape and 3 classes
)

test_generator_custom = tf.data.Dataset.from_generator(
    lambda: dual_input_generator(test_generator),
    output_signature=get_output_signature((176, 208, 3), 3)
)


In [ ]:
history = model.fit(
    train_generator_custom,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator_custom,
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=100 # Adjust as needed
)


Epoch 1/100


C:\Users\DELL\.conda\envs\ali\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_8']. Received: the structure of inputs=('*', '*')
  warnings.warn(


202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5284 - loss: 1.0151

C:\Users\DELL\.conda\envs\ali\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_8']. Received: the structure of inputs=('*', '*')
  warnings.warn(


202/202 ━━━━━━━━━━━━━━━━━━━━ 1919s 9s/step - accuracy: 0.5288 - loss: 1.0138 - val_accuracy: 0.6272 - val_loss: 0.6366
Epoch 2/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 1795s 9s/step - accuracy: 0.6876 - loss: 0.6007 - val_accuracy: 0.7171 - val_loss: 0.5426
Epoch 3/100
141/202 ━━━━━━━━━━━━━━━━━━━━ 7:31 7s/step - accuracy: 0.7152 - loss: 0.5711

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_generator_custom, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper left')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Step 1: Get the true labels and predictions from the test generator
true_labels = []
pred_labels = []

for images, labels in test_generator_custom:
    predictions = model.predict(images)
    
    # Convert predictions and true labels to class indices
    pred_labels.extend(np.argmax(predictions, axis=1))
    true_labels.extend(np.argmax(labels.numpy(), axis=1))
    
    # Stop when we have gone through the entire test dataset
    if len(true_labels) >= test_generator.samples:
        break

# Step 2: Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)

# Step 3: Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=test_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()
